In [ ]:
import numpy as np
import smtplib
import imghdr
from email.message import EmailMessage

In [ ]:
def my_label(image_name):
    name = image_name.split('.')[-3] 

    
    
    if name=="SaiDinesh":
        return np.array([1,0])
    else:
        return np.array([0,1])

In [ ]:
import cv2
import os
from random import shuffle
from tqdm import tqdm

In [ ]:
def my_data():
    data = []
    for img in tqdm(os.listdir("Images")):
        path=os.path.join("Images",img)
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        data.append([np.array(img_data), my_label(img)])
    shuffle(data)  
    return data

In [ ]:
data = my_data()


In [ ]:
train = data[:1620]  
test = data[1620:]
X_train = np.array([i[0] for i in train]).reshape(-1,50,50,1)
print(X_train.shape)
y_train = [i[1] for i in train]
X_test = np.array([i[0] for i in test]).reshape(-1,50,50,1)
print(X_test.shape)
y_test = [i[1] for i in test]

In [ ]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
convnet = input_data(shape=[50,50,1])
convnet = conv_2d(convnet, 32, 5, activation='relu')
# 32 filters and stride=5 so that the filter will move 5 pixel or unit at a time
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate = 0.001, loss='categorical_crossentropy')
model = tflearn.DNN(convnet, tensorboard_verbose=1)
model.fit(X_train, y_train, n_epoch=100, validation_set=(X_test, y_test), show_metric = True, run_id="FRS" )

In [ ]:
def data_for_visualization():
    Vdata = []
    for img in tqdm(os.listdir("Images")):
        path = os.path.join("Images", img)
        img_num = img.split('.')[0] 
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        Vdata.append([np.array(img_data), img_num])
    shuffle(Vdata)
    return Vdata

In [ ]:
Vdata = data_for_visualization()

In [ ]:
import matplotlib.pyplot as plt   # pip install matplotlib

fig = plt.figure(figsize=(20,20))
for num, data in enumerate(Vdata[:20]):
    img_data = data[0]
    y = fig.add_subplot(5,5, num+1)
    image = img_data
    data = img_data.reshape(50,50,1)
    model_out = model.predict([data])[0]
    print(model_out)
    if np.argmax(model_out) == 0:
        my_label = 'Sai Dinesh'
    else:
        my_label = 'Sri Pardhiv'
        
    y.imshow(image, cmap='gray')
    plt.title(my_label)
    
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

In [ ]:
face_detect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
def sendmail(image_name):
    Sender_Email = "neikint5435@gmail.com"
    Reciever_Email = "neikint5435@gmail.com"
    Password = 'Saigeetha@22'

    newMessage = EmailMessage()                         
    newMessage['Subject'] = "Intruder alert" 
    newMessage['From'] = Sender_Email                   
    newMessage['To'] = Reciever_Email                   
    newMessage.set_content('Someone is entered!') 

    with open('sai.jpg', 'rb') as f:
        image_data = f.read()
        image_type = imghdr.what(f.name)
        image_name = f.name

    newMessage.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(Sender_Email, Password)              
        smtp.send_message(newMessage)

In [ ]:
from playsound import playsound

In [ ]:
def sound(Text):
    try:
        playsound('Alert123.mp3')
    except:
        print("error occured")

In [ ]:
source = cv2.VideoCapture(0)
while 1:
    not_to_use, image = source.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detect.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        face_roi = gray[y:y+h,x:x+w]
        resized_image = cv2.resize(face_roi, (200, 200))
        reshaped_face = np.reshape(resized_image, (-1,50,50,1))
        result = model.predict(reshaped_face)[0]
        print(result , result[0] , result[1])
        if result[0] >  0.69:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, "Innocent", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(image, "Intruder", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            sendmail(image)
            Text = 'Alert sound'
            sound(Text)
            
    cv2.imshow('Verzeo Project', image)
    key = cv2.waitKey(1)
    if key == 27:
        break
cv2.destroyAllWindows()
source.release()